# Kindlmann Color Map

This iPython notebook contains the script required to derive what is often known as the Kindlmann color map, so named because its first known design is in [a paper by Kindlmann, Reinhard, and Creem](http://www.cs.utah.edu/~gk/papers/vis02/), although this derivation takes inspiration from other sources. The map is basically the rainbow color map with the luminance adjusted such that it monotonically changes, making it much more perceptually viable.

This code relies on the [python-colormath](http://python-colormath.readthedocs.org/en/latest/index.html) module. See [its documentation](http://python-colormath.readthedocs.org/en/latest/index.html) for information such as installation instructions. (It can be installed with either pip or macports.)

In [288]:
from colormath.color_objects import *
from colormath.color_conversions import convert_color

Mostly because it's habit, I am also using [pandas](http://pandas.pydata.org/) dataframes to organize the data. (Pandas can be installed with macports.)

In [289]:
import pandas
import numpy

We will also be using [toyplot](https://toyplot.readthedocs.org) for making visuals. See its documentation for installation instructions.

In [290]:
import toyplot
import toyplot.pdf

### Support Functions

The original Kindlmann paper created their color map by having a human user adjust the brightness to a prescribed luminance. (The color map was really proposed as a use case for a technique that allows humans to match colors.) Personally, I think this technique is overkill. First of all, no one is going to calibrate to their display. (*I* don't even want to do it one time just to get initial values.) Second, any change in the display will invalidate the calibration anyway.

Instead, I am going to use a programmatic technique proposed in [a blog post by Matteo Niccoli](https://mycarta.wordpress.com/2012/12/06/the-rainbow-is-deadlong-live-the-rainbow-part-5-cie-lab-linear-l-rainbow/). The idea is to convert the RGB values to the perceptual CIELAB space, adjust the L (luminance) value in CIELAB, and then convert back to RGB.

To do this, we need a function that takes a hue value and adjusts its luminance. Of course, we need to convert to CIELAB and alter the L value. But then we also have to adjust the a and b values so that the color is back in the representable color gamut. We do this with a binary search.

In [291]:
def valid_color(color):
    '''Given a color from the colormath.color_objects package,
    returns whether it can be displayed in RGB.'''
    rgb = convert_color(color, sRGBColor).get_upscaled_value_tuple()
    return ((rgb[0] >= 0) and (rgb[0] <= 255) and
            (rgb[1] >= 0) and (rgb[1] <= 255) and
            (rgb[2] >= 0) and (rgb[2] <= 255))

def safe_color(color):
    '''Given a color from the colormath.color_objects package,
    returns whether it is in the RGB color gamut and far enough
    away from the gamut border to be considered 'safe.' Colors
    right on the edge of displayable colors sometimes do not
    display quite right and also sometimes leave the color
    gamut when interpolated.'''
    rgb_color = convert_color(color, sRGBColor)
    rgb_vector = rgb_color.get_value_tuple()
    clamp_dist = 0.05*(numpy.max(rgb_vector) - numpy.min(rgb_vector))
    return ((rgb_color.rgb_r >= clamp_dist) and (rgb_color.rgb_r <= 1-clamp_dist) and
            (rgb_color.rgb_g >= clamp_dist) and (rgb_color.rgb_g <= 1-clamp_dist) and
            (rgb_color.rgb_b >= clamp_dist) and (rgb_color.rgb_b <= 1-clamp_dist))

def scale_hue(hue, scalar):
    '''Given a hue value (in degrees) and a scalar value between
    0 and 1, create a color to have a luminance proportional to
    the scalar with the given hue. Returns an sRGBColor value.'''
    #Special cases
    if scalar <= 0:
        return sRGBColor(0, 0, 0)
    if scalar >= 1:
        return sRGBColor(1, 1, 1)

    hsv_original = HSVColor(hue, 1.0, 1.0)
    rgb_original = convert_color(hsv_original, sRGBColor)
    lab_original = convert_color(rgb_original, LabColor)
    l_target = 100.0*scalar
    a_original = lab_original.lab_a
    b_original = lab_original.lab_b
    
    high_scale = 1.0
    low_scale = 0.0
    for i in xrange(0, 12):
        mid_scale = (high_scale-low_scale)/2 + low_scale
        if safe_color(LabColor(l_target, mid_scale*a_original, mid_scale*b_original)):
            low_scale = mid_scale
        else:
            high_scale = mid_scale
            
    return convert_color(LabColor(l_target, low_scale*a_original, low_scale*b_original),
                         sRGBColor)

A convenience function that takes a column of RGB triples in a pandas dataframe, unzips it, and adds three columns to the data frame with the red, green, and blue values.

In [305]:
def unzip_rgb_triple(dataframe, column='RGB'):
    '''Given a dataframe and the name of a column holding an RGB triplet,
    this function creates new separate columns for the R, G, and B values
    with the same name as the original with '_r', '_g', and '_b' appended.'''
    # Creates a data frame with separate columns for the triples in the given column
    unzipped_rgb = pandas.DataFrame(dataframe[column].values.tolist(),
                                    columns=['r', 'g', 'b'])
    # Add the columns to the original data frame
    dataframe[column + '_r'] = unzipped_rgb['r']
    dataframe[column + '_g'] = unzipped_rgb['g']
    dataframe[column + '_b'] = unzipped_rgb['b']

### Original Color Map

The color map in the original Kindlmann paper follows the rainbow hues from purple through blue and green to red. We will use those hues first, but also derive a slightly alternate one below.

In [298]:
start_hue = 300.0
end_hue = 0.0

We start by creating a "short" map with a minimal amount of control points. These control points are placed where the RGB interpolation bends, which every 60 degrees in HSV space. Create a table starting with the scalar values and the hue angle for each one.

In [299]:
data_short = pandas.DataFrame()
data_short['hue'] = numpy.arange(start_hue, end_hue-0.0001, -30.0)
data_short['scalar'] = numpy.linspace(0.0, 1.0, data_short['hue'].size)
data_short

,hue,scalar
0,300,0.0
1,270,0.1
2,240,0.2
3,210,0.3
4,180,0.4
5,150,0.5
6,120,0.6
7,90,0.7
8,60,0.8
9,30,0.9


Use the `scale_hue` function on each row to get the color we should use at each point.

In [300]:
color_array = data_short.apply(lambda row: scale_hue(row['hue'], row['scalar']),
                               axis=1)
data_short['sRGBColor'] = color_array
data_short['RGB'] = color_array.apply(lambda rgb: rgb.get_upscaled_value_tuple())
data_short['sRGB'] = color_array.apply(lambda rgb: rgb.get_value_tuple())
data_short

,hue,scalar,sRGBColor,RGB,sRGB
0,300,0.0,sRGBColor (rgb_r:0.0000 rgb_g:0.0000 rgb_b:0.0...,"(0, 0, 0)","(0.0, 0.0, 0.0)"
1,270,0.1,sRGBColor (rgb_r:0.1796 rgb_g:0.0143 rgb_b:0.2...,"(46, 4, 76)","(0.179639485725, 0.0143460026832, 0.299732434447)"
2,240,0.2,sRGBColor (rgb_r:0.2481 rgb_g:0.0271 rgb_b:0.5...,"(63, 7, 145)","(0.2481175218, 0.0271274399767, 0.569636574265)"
3,210,0.3,sRGBColor (rgb_r:0.0311 rgb_g:0.2589 rgb_b:0.6...,"(8, 66, 165)","(0.0311312658436, 0.258880996016, 0.646266425307)"
4,180,0.4,sRGBColor (rgb_r:0.0199 rgb_g:0.4159 rgb_b:0.4...,"(5, 106, 106)","(0.0198728054351, 0.415899382496, 0.415537874164)"
5,150,0.5,sRGBColor (rgb_r:0.0259 rgb_g:0.5384 rgb_b:0.2...,"(7, 137, 69)","(0.0258836453499, 0.538384409006, 0.269670037362)"
6,120,0.6,sRGBColor (rgb_r:0.0315 rgb_g:0.6581 rgb_b:0.1...,"(8, 168, 26)","(0.0315354727534, 0.658121193765, 0.103322272239)"
7,90,0.7,sRGBColor (rgb_r:0.3311 rgb_g:0.7613 rgb_b:0.0...,"(84, 194, 9)","(0.331114702761, 0.761347803523, 0.036567067771)"
8,60,0.8,sRGBColor (rgb_r:0.7682 rgb_g:0.8091 rgb_b:0.0...,"(196, 206, 10)","(0.768180014053, 0.809064201855, 0.0395103035395)"
9,30,0.9,sRGBColor (rgb_r:0.9891 rgb_g:0.8620 rgb_b:0.7...,"(252, 220, 197)","(0.989135834421, 0.862040574175, 0.772835569979)"


Now repeat creating this table but for a much higher resolution.

In [301]:
data_long = pandas.DataFrame()
data_long['hue'] = numpy.linspace(start_hue, end_hue, 1024)
data_long['scalar'] = numpy.linspace(0.0, 1.0, 1024)

color_array = data_long.apply(lambda row: scale_hue(row['hue'], row['scalar']),
                              axis=1)
data_long['sRGBColor'] = color_array
data_long['RGB'] = color_array.apply(lambda rgb: rgb.get_upscaled_value_tuple())
data_long['sRGB'] = color_array.apply(lambda rgb: rgb.get_value_tuple())

Use toyplot to make a visual representation of the color map.

In [302]:
kindlmann_palette = toyplot.color.Palette(colors=data_long['sRGB'].values)
kindlmann_map = toyplot.color.LinearMap(palette=kindlmann_palette)
#kindlmann_map

In [303]:
canvas = toyplot.Canvas(width=512, height=200)
axes = canvas.axes()
axes.scatterplot(data_short['scalar'], numpy.zeros(data_short['scalar'].shape),
                 marker='|', size=1000,
                 color='gray')
axes.scatterplot(data_long['scalar'], numpy.zeros(data_long['scalar'].shape),
                 marker='|', size=500,
                 color=kindlmann_map)
axes.y.show = False
axes.x.show = False

for index in xrange(0, len(data_short.index), 2):
    axes.text(data_short['scalar'][index], 0,
              '%1.1f' % data_short['scalar'][index],
              style={'baseline-shift':'-23px',
                     'font-size': '8pt'},
              color='black')
    axes.text(data_short['scalar'][index], 0,
              str(data_short['RGB'][index]),
              style={'baseline-shift':'-34px',
                     'font-size':'7pt'},
              color='black')

for index in xrange(1, len(data_short.index), 2):
    axes.text(data_short['scalar'][index], 0,
              '%1.1f' % data_short['scalar'][index],
              style={'baseline-shift':'34px',
                     'font-size': '8pt'},
              color='black')
    axes.text(data_short['scalar'][index], 0,
              str(data_short['RGB'][index]),
              style={'baseline-shift':'23px',
                     'font-size':'7pt'},
              color='black')

In [304]:
toyplot.pdf.render(canvas, 'KindlmannMap.pdf')

In [309]:
unzip_rgb_triple(data_long, 'RGB')
data_long.to_csv('KindlmannUChar.csv',
                 index=False,
                 columns=['scalar', 'RGB_r', 'RGB_g', 'RGB_b'])

unzip_rgb_triple(data_long, 'sRGB')
data_long.to_csv('KindlmannFloat.csv',
                 index=False,
                 columns=['scalar', 'sRGB_r', 'sRGB_g', 'sRGB_b'],
                 header=['scalar', 'RGB_r', 'RGB_g', 'RGB_b'])

### Extended Hue Color Map

The original Kindlmann color map starts at purple and ends a red, but it is difficult to see the colors at each end. There is no reason why we can't extend the map, in particular to see the reds better. I doubt there is more perceptual information, but it might look prettier.

The hues we use are similar to the ones suggested by Dave Green for his [cubehelix](https://www.mrao.cam.ac.uk/~dag/CUBEHELIX/). The cubehelix approach does computations in the RGB color space, which is easier but not as perceptually uniform. We will repeat the same computations as above but use hues closer to what Green used. The algorithm here will be both perceptually uniform and in some places produces more vibrant hues (so we use slightly less rotation).

In [310]:
start_hue = 300.0
end_hue = -180.0

We start by creating a "short" map with a minimal amount of control points. These control points are placed where the RGB interpolation bends, which every 60 degrees in HSV space. Create a table starting with the scalar values and the hue angle for each one.

In [312]:
data_short = pandas.DataFrame()
data_short['hue'] = numpy.arange(start_hue, end_hue-0.0001, -30.0)
data_short['scalar'] = numpy.linspace(0.0, 1.0, data_short['hue'].size)

Use the `scale_hue` function on each row to get the color we should use at each point.

In [313]:
color_array = data_short.apply(lambda row: scale_hue(row['hue'], row['scalar']),
                               axis=1)
data_short['sRGBColor'] = color_array
data_short['RGB'] = color_array.apply(lambda rgb: rgb.get_upscaled_value_tuple())
data_short['sRGB'] = color_array.apply(lambda rgb: rgb.get_value_tuple())

Now repeat creating this table but for a much higher resolution.

In [314]:
data_long = pandas.DataFrame()
data_long['hue'] = numpy.linspace(start_hue, end_hue, 1024)
data_long['scalar'] = numpy.linspace(0.0, 1.0, 1024)

color_array = data_long.apply(lambda row: scale_hue(row['hue'], row['scalar']),
                              axis=1)
data_long['sRGBColor'] = color_array
data_long['RGB'] = color_array.apply(lambda rgb: rgb.get_upscaled_value_tuple())
data_long['sRGB'] = color_array.apply(lambda rgb: rgb.get_value_tuple())

Use toyplot to make a visual representation of the color map.

In [302]:
kindlmann_green_palette = toyplot.color.Palette(colors=data_long['sRGB'].values)
kindlmann_green_map = toyplot.color.LinearMap(palette=kindlmann_palette)
#kindlmann_map

In [316]:
canvas = toyplot.Canvas(width=512, height=200)
axes = canvas.axes()
#axes.scatterplot(data_short['scalar'], numpy.zeros(data_short['scalar'].shape),
#                 marker='|', size=1000,
#                 color='gray')
axes.scatterplot(data_long['scalar'], numpy.zeros(data_long['scalar'].shape),
                 marker='|', size=500,
                 color=kindlmann_map)
axes.y.show = False
axes.x.show = False

#for index in xrange(0, len(data_short.index), 2):
#    axes.text(data_short['scalar'][index], 0,
#              '%1.1f' % data_short['scalar'][index],
#              style={'baseline-shift':'-23px',
#                     'font-size': '8pt'},
#              color='black')
#    axes.text(data_short['scalar'][index], 0,
#              str(data_short['RGB'][index]),
#              style={'baseline-shift':'-34px',
#                     'font-size':'7pt'},
#              color='black')

#for index in xrange(1, len(data_short.index), 2):
#    axes.text(data_short['scalar'][index], 0,
#              '%1.1f' % data_short['scalar'][index],
#              style={'baseline-shift':'34px',
#                     'font-size': '8pt'},
#              color='black')
#    axes.text(data_short['scalar'][index], 0,
#              str(data_short['RGB'][index]),
#              style={'baseline-shift':'23px',
#                     'font-size':'7pt'},
#              color='black')

In [317]:
toyplot.pdf.render(canvas, 'KindlmannGreenMap.pdf')

In [318]:
unzip_rgb_triple(data_long, 'RGB')
data_long.to_csv('KindlmannGreenUChar.csv',
                 index=False,
                 columns=['scalar', 'RGB_r', 'RGB_g', 'RGB_b'])

unzip_rgb_triple(data_long, 'sRGB')
data_long.to_csv('KindlmannGreenFloat.csv',
                 index=False,
                 columns=['scalar', 'sRGB_r', 'sRGB_g', 'sRGB_b'],
                 header=['scalar', 'RGB_r', 'RGB_g', 'RGB_b'])